In [1]:
import folium
import pandas as pd
import h3
from shapely.geometry import Polygon, Point
import shapely.wkt
from geojson import Point, Feature, FeatureCollection, dump

# Create a GeoJson

### First find all used hexagons

In [2]:
#Get this file from scibo
df = pd.read_csv('sharenow_supply_data_6mo.csv')

df_StartEnd = df[['hex_start_7','hex_end_7']]

list_StartUnique = list(df['hex_start_7'].unique())
list_EndUnique = list(df['hex_end_7'].unique())
list_UniqueHexs = list(pd.Series(list_StartUnique+list_EndUnique).unique())

### Now get the cornerstones

In [3]:
def output_h3_id_attributes(h3_id):
    return Polygon(h3.h3_to_geo_boundary(h3_id, geo_json=True))

In [4]:
features = []
for hexID in list_UniqueHexs:
    features.append(Feature(geometry=output_h3_id_attributes(hexID), properties={"hexID": hexID}))

feature_collection = FeatureCollection(features)

with open('berlin_charger.geojson', 'w') as f:
   dump(feature_collection, f)

# Visulise

In [5]:
map = folium.Map(location=[52.518839, 13.399304], default_zoom_start=30)

In [6]:
chargerPerHex = pd.read_csv('charger_in_hex.csv')
finalData = chargerPerHex[['hexID']]
chargerPerHex = chargerPerHex[['x1','x2','x3']].sum(axis=1)
finalData['charger'] = chargerPerHex

finalData.head(5)

,hexID,charger
0,871f1d4d0ffffff,0.0
1,871f1d48affffff,0.0
2,871f1d48effffff,0.0
3,871f1d499ffffff,5.0
4,871f18b25ffffff,0.0


In [7]:
map.choropleth(geo_data="berlin_charger.geojson",
             data=finalData, 
             columns=['hexID', 'charger'], 
             key_on='feature.properties.hexID', 
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Charger')
map